In [1]:
#pip install requests msal

In [2]:
import json
import msal
import os
import requests
import secrets
import uuid
import webbrowser
from http.server import BaseHTTPRequestHandler, HTTPServer
import base64
import hashlib

In [87]:
# Replace the following placeholders with your actual values
CLIENT_ID = 'xxxxxxxxxxxxxx'
CLIENT_SECRET = 'xxxxxxxxxxxxxxxxxxx'
TENANT_ID = 'xxxxxxxxxxxxxxxxxxx'
USER_ID = 'xxxxxxxxxxxxxxxxxxx'
FILE_DIR = '/home/rmsryu/notebooks/CA2/ca2-twitter/notebooks/data'
FILE_PATH = f'{FILE_DIR}/README.txt'
REDIRECT_PORT = 8080  # Choose a port for the local server

# Create the MSAL app
app = msal.PublicClientApplication(
    CLIENT_ID,
    authority=f'https://login.microsoftonline.com/{TENANT_ID}',
)

def urlsafe_b64encode_no_padding(data):
    return base64.urlsafe_b64encode(data).rstrip(b'=').decode('utf-8')

# Define the PKCE parameters
code_verifier = secrets.token_urlsafe(128)[:128]

# Compute the SHA-256 hash
hash_object = hashlib.sha256(code_verifier.encode('utf-8'))
code_challenge = urlsafe_b64encode_no_padding(hash_object.digest())

# Generate the authorization request URL
auth_request_url = app.get_authorization_request_url(
    ["https://graph.microsoft.com/Files.ReadWrite"],
    redirect_uri=f"http://localhost:{REDIRECT_PORT}",
    state=str(uuid.uuid4()),
    prompt="select_account",
    code_challenge=code_challenge,
    code_challenge_method="S256",
)

# Open the authorization request URL in the user's default browser
print(f"Open this link and accept consent: {auth_request_url}")


Open this link and accept consent: https://login.microsoftonline.com/52a239cb-7253-407a-8d71-74c8efe4d804/oauth2/v2.0/authorize?client_id=b7b2854f-7be5-46e3-8629-6f1cee4ec9fc&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8080&scope=https%3A%2F%2Fgraph.microsoft.com%2FFiles.ReadWrite+offline_access+openid+profile&state=9101fb58-7e89-4734-9abe-5d5c3f9cebcc&prompt=select_account


In [64]:
# Copy the auth_code - remove &state=007415da-b1d4-4ed5-a68b-e9d3305e15a1 &session_state=8ca68bd3-8cf7-42ac-9ac3-b2e5ff8ec9e3#
auth_code="0.ASEAyzmiUlNyekCNcXTI7-TYBE-Fsrfle-NGhilvHO5OyfyGACw.AgABAAIAAAD--DLA3VO7QrddgJg7WevrAgDs_wUA9P-KC4SLiXeb8VD4LcYO6jqnLLJ2OYFMR60GQIeftK90LRJQyYHJ1z6IgfWFyFvv0lFJOA66RlIKFm_fkFthsBGnBqoR40reuYeuFTpeUv2n9dCkrg7d5G134jAVZVfqpDHn18mrxvxFqemrER8BLJIF-C5S6mIu8wcZDtJld_Hwt3L68p1tryMDpdJB0_Qz5j8fxthn_rb7943OcgifTP1heCs5nUCl4hXJmzR7IhWi7Hgef2LaAxrwoGBeeMQDTXEX31F5P23JytN0dGE0bmOUc0QHsu0tDyUnjyBO65OCixkS6OPMSqPaF8xX3xW_2J3mi6S48qmOQ9y0ESWysAlMuGJQVT8rAXfUo3AlxaYyCMK7jFfxYDp_FbVGDaA6Jht4Q1sD_LIHeU31roudipllcvDVC6FYjuJrkBhaYFqFq_-iXG2mG-BFHhRP7JNEN6nVTa_Y29AhRR3DPuvh7RHWgFZL1fEQqlO7p8xeQ9oDotpRcLOliVVRvJjiG4FVPprF3n6VN3E_nIifWGR-n98usoo9tyVqRYd_bTdKYJyncW1qyfaOpm-g5LjRJWKPmJZX-sm85iqosoSgYPqtz1SNCAHC1iwrPu3ylfMLrSzRU0vAxXedA-14sb9zoyQTpYRNa9sM04nYgMrgZ169j_9sdWIbjSd4CzJ8HaSPU7O-ksrO-CtaDzYnE-FA9NREZyyvYSnnImrEQkinNFucS7NQNdBSh4ohh10hImNGagrCha0cDQ"

In [65]:
#https://learn.microsoft.com/en-us/azure/active-directory/develop/v2-oauth2-auth-code-flow
def get_oauth_token():
    # Get access token
    url = f'https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token'
    token_data = {
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'code': auth_code,
        'redirect_uri': f"http://localhost:{REDIRECT_PORT}",
        'grant_type': 'authorization_code'
    }
    token_response = requests.post(url, data=token_data)
    print(token_response)
    return token_response.json()['access_token']

In [ ]:
access_token = get_oauth_token()

In [68]:
access_token = "eyJ0eXAiOiJKV1QiLCJub25jZSI6Imx6cTdiOXJPYkJpSUR3RnVyUFdVZWRUaXExVDk5cENVTk84dmVMUU1nVUkiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC81MmEyMzljYi03MjUzLTQwN2EtOGQ3MS03NGM4ZWZlNGQ4MDQvIiwiaWF0IjoxNjgyNjM1NDczLCJuYmYiOjE2ODI2MzU0NzMsImV4cCI6MTY4MjY0MDcyMywiYWNjdCI6MCwiYWNyIjoiMSIsImFpbyI6IkFUUUF5LzhUQUFBQThuSnJsWEx2MXdTM1FHcGtCak1xUU9EM1lnSFZ6TmtxRWNia1BPZGt6ZUExci9BeTFxT3h0bVhJcnk2M2JabEUiLCJhbXIiOlsicHdkIl0sImFwcF9kaXNwbGF5bmFtZSI6InNiczIyMDIxLXR3aXR0ZXItZGF0YSIsImFwcGlkIjoiYjdiMjg1NGYtN2JlNS00NmUzLTg2MjktNmYxY2VlNGVjOWZjIiwiYXBwaWRhY3IiOiIxIiwiZmFtaWx5X25hbWUiOiJTYW5jaGV6IiwiZ2l2ZW5fbmFtZSI6IlJhdWwgTWFydGluIiwiaWR0eXAiOiJ1c2VyIiwiaXBhZGRyIjoiODYuNDEuMTc0LjM2IiwibmFtZSI6IlJhdWwgTWFydGluIFNhbmNoZXoiLCJvaWQiOiI1MDg3MmM2My01YTg4LTQ5NzQtODhmMi0wZGRkYTU4ODY1NDQiLCJwbGF0ZiI6IjMiLCJwdWlkIjoiMTAwMzIwMDFEQzI2MzRCRSIsInJoIjoiMC5BU0VBeXptaVVsTnlla0NOY1hUSTctVFlCQU1BQUFBQUFBQUF3QUFBQUFBQUFBQ0dBQ3cuIiwic2NwIjoiRmlsZXMuUmVhZFdyaXRlIG9wZW5pZCBwcm9maWxlIGVtYWlsIiwic2lnbmluX3N0YXRlIjpbImttc2kiXSwic3ViIjoiSHZfV0xoa2Y2cmFoNXpidHpyck8xekRyS3VHYUFBcGVqdmdFLXljQkdiNCIsInRlbmFudF9yZWdpb25fc2NvcGUiOiJFVSIsInRpZCI6IjUyYTIzOWNiLTcyNTMtNDA3YS04ZDcxLTc0YzhlZmU0ZDgwNCIsInVuaXF1ZV9uYW1lIjoic2JzMjIwMjFAc3R1ZGVudC5jY3QuaWUiLCJ1cG4iOiJzYnMyMjAyMUBzdHVkZW50LmNjdC5pZSIsInV0aSI6InJSb0cwMkYyZVVpSGlWWk9EZU13QUEiLCJ2ZXIiOiIxLjAiLCJ3aWRzIjpbImI3OWZiZjRkLTNlZjktNDY4OS04MTQzLTc2YjE5NGU4NTUwOSJdLCJ4bXNfc3QiOnsic3ViIjoiTVVQdTBnLU9GYTQyLURlMEZZNnUyM1F6bWVCXzdSck1wT3ZMQm9vWENxbyJ9LCJ4bXNfdGNkdCI6MTQ3ODExNDI1NiwieG1zX3RkYnIiOiJFVSJ9.iUggkqydMqDRRMyUljvQndwK2ZdeZrkrNdKU8s4FYoWrFGb4WX-f71ECvSK7SyWNM9Nck48gfdt1CmAN7vCMhll9pqArllOCWy_DB9cZA_dACbktF4l-mkj_kl-GEBd_XNwmk4MzBtG9zNs7zpvNnFPHUfFlOIgY9ginRG04aF6z0ulh_wLGGjBaa2p91EmgVmHjTDDgfg9LQZlIbX9Mr0ODSit_ZDkIe8Oi7zwh8oWdBvKaMj6UkVZyrxGopNcAyw8KczgrTO24_kgqveZ866C53d-Nf3Q_rug_VbE5vT_zZZ-9zqzBdrUsXLY5x21lYwPHq6GH-A8A5iuHS2NTQw"
print(access_token)

eyJ0eXAiOiJKV1QiLCJub25jZSI6Imx6cTdiOXJPYkJpSUR3RnVyUFdVZWRUaXExVDk5cENVTk84dmVMUU1nVUkiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC81MmEyMzljYi03MjUzLTQwN2EtOGQ3MS03NGM4ZWZlNGQ4MDQvIiwiaWF0IjoxNjgyNjM1NDczLCJuYmYiOjE2ODI2MzU0NzMsImV4cCI6MTY4MjY0MDcyMywiYWNjdCI6MCwiYWNyIjoiMSIsImFpbyI6IkFUUUF5LzhUQUFBQThuSnJsWEx2MXdTM1FHcGtCak1xUU9EM1lnSFZ6TmtxRWNia1BPZGt6ZUExci9BeTFxT3h0bVhJcnk2M2JabEUiLCJhbXIiOlsicHdkIl0sImFwcF9kaXNwbGF5bmFtZSI6InNiczIyMDIxLXR3aXR0ZXItZGF0YSIsImFwcGlkIjoiYjdiMjg1NGYtN2JlNS00NmUzLTg2MjktNmYxY2VlNGVjOWZjIiwiYXBwaWRhY3IiOiIxIiwiZmFtaWx5X25hbWUiOiJTYW5jaGV6IiwiZ2l2ZW5fbmFtZSI6IlJhdWwgTWFydGluIiwiaWR0eXAiOiJ1c2VyIiwiaXBhZGRyIjoiODYuNDEuMTc0LjM2IiwibmFtZSI6IlJhdWwgTWFydGluIFNhbmNoZXoiLCJvaWQiOiI1MDg3MmM2My01YTg4LTQ5NzQtODhmMi0wZGRkYTU4ODY1NDQiLCJwbGF0ZiI6IjMiLCJwdWlkIjoiMTAwMzIwMDFEQzI2MzRCRSIsInJoIjoiMC5BU0VBeXp

In [94]:
def get_all_files(directory):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list


def upload_large_file_in_chunks(access_token, file_path):
    filename = os.path.basename(file_path)

    # Create an upload session for the large file
    create_session_url = f"https://graph.microsoft.com/v1.0/me/drive/root:/twitter/{filename}:/createUploadSession"
    create_session_data = {
        "item": {
            "@microsoft.graph.conflictBehavior": "replace"
        }
    }
    headers = {
        "Authorization": f"Bearer {access_token}",
    }
    create_session_response = requests.post(create_session_url, headers=headers, json=create_session_data)
    upload_url = create_session_response.json()["uploadUrl"]

    # Upload the file in chunks
    chunk_size = 320 * 1024  # 320 KB
    file_size = os.path.getsize(file_path)
    with open(file_path, "rb") as file:
        for start in range(0, file_size, chunk_size):
            end = min(start + chunk_size - 1, file_size - 1)
            file.seek(start)
            chunk_data = file.read(chunk_size)
            headers = {
                "Content-Length": str(end - start + 1),
                "Content-Range": f"bytes {start}-{end}/{file_size}",
            }
            chunk_response = requests.put(upload_url, headers=headers, data=chunk_data)

            if chunk_response.status_code not in [200, 201, 202]:
                print(f"Error uploading chunk from {start} to {end}")
                return

    print(f"File '{filename}' uploaded successfully")

def upload_to_one_drive(access_token, file_path):
    # Upload the file
    filename = os.path.basename(file_path)
    
    check_file_url = f"https://graph.microsoft.com/v1.0/me/drive/root:/twitter/{filename}"
    headers = {
        "Authorization": f"Bearer {access_token}",
    }
    # Check if file exists
    check_response = requests.get(check_file_url, headers=headers)

    if check_response.status_code == 200:
        print(f"The file '{filename}' already exists in your OneDrive.")
    else:
        upload_large_file_in_chunks(access_token,file_path)


In [95]:
for file in get_all_files(FILE_DIR):
    upload_to_one_drive(access_token,file)


File 'twitter_stream_2020_01_01.tar' uploaded successfully


KeyError: 'uploadUrl'

In [96]:
create_session_response

NameError: name 'create_session_response' is not defined